# Narrative Prospective Lab (Game-Aligned)

Experimental analysis space that remains trait-name aligned with game assets.


In [ ]:
import sys
from pathlib import Path

for base in [Path.cwd(), Path.cwd().parent]:
    if (base / 'src' / 'dungeonbreak_narrative').exists():
        sys.path.insert(0, str(base))
        break

from dungeonbreak_narrative import (
    load_game_traits_manifest,
    load_narrative_snapshot,
    validate_game_alignment_warn_only,
    vector_from_trait_map,
)

import numpy as np
import plotly.graph_objects as go


In [ ]:
alignment = validate_game_alignment_warn_only()
for warning in alignment['warnings']:
    print('WARN:', warning)

traits = load_game_traits_manifest()
snapshot = load_narrative_snapshot()

entity_vectors = []
for entity in snapshot.get('entities', []):
    start_map = entity.get('starting_coordinates') or {}
    if start_map:
        entity_vectors.append((entity.get('name', 'unknown'), vector_from_trait_map(start_map, trait_names=traits)))

if len(entity_vectors) < 2:
    print('Data unavailable: snapshot needs at least two entities with starting_coordinates for prospective projection.')
else:
    names = [name for name, _ in entity_vectors]
    X = np.array([v for _, v in entity_vectors], dtype=float)
    Xc = X - X.mean(axis=0, keepdims=True)
    U, S, Vt = np.linalg.svd(Xc, full_matrices=False)
    XY = Xc @ Vt[:2].T

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=XY[:, 0], y=XY[:, 1], mode='markers+text', text=names, textposition='top center'))
    fig.update_layout(title='Entity projection (SVD over game trait vectors)', xaxis_title='component_1', yaxis_title='component_2')
    fig.show()


## Optional text-embedding experiment


In [ ]:
try:
    from sentence_transformers import SentenceTransformer  # type: ignore
except Exception:
    print('sentence-transformers not installed. Optional install: uv sync --extra embeddings')
else:
    labels = [d.get('label', '') for d in snapshot.get('dialogs', []) if d.get('label')]
    if not labels:
        print('No dialog labels found for embedding probe.')
    else:
        model = SentenceTransformer('all-MiniLM-L6-v2')
        emb = model.encode(labels)
        print('Embedding matrix shape:', emb.shape)
        norms = np.linalg.norm(emb, axis=1)
        for label, norm in zip(labels, norms):
            print({'label': label, 'norm': float(norm)})
